# Estudiante : Jeisson Estevens Araque Ramirez
# Asignatura : Programación para análisis de datos
# Docente: Diego Leon Zapata Ruiz
### Evidencia de aprendizaje 1. Análisis y herramientas de extracción de datos
--- 

In [1]:
# Importar las librerias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Web scraping con selenium

In [ ]:
opts = Options()

opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")
# Configuración para no abrir el navegador
opts.add_argument("--headless")


# Configurar el controlador web
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=opts
) 

# Abre la página de Mercado Libre
driver.get("https://listado.mercadolibre.com.co/")

# Espera a que se cargue la página
time.sleep(3)

# Buscar "smart watch" en la barra de búsqueda
print("Buscando en el navegador")
search_bar = driver.find_element(By.NAME, "as_word")
search_bar.send_keys("smart-watch")
search_bar.send_keys(Keys.RETURN)

# Espera a que se carguen los resultados
time.sleep(3)

# Lista para almacenar los datos
watch_data = []

# Extrae los primeros resultados de "smart watch"
print("Extrayendo resultados")
products = driver.find_elements(By.XPATH, "//li[contains(@class, 'ui-search-layout__item')]")
for product in products[:20]:  # Limitar a los primeros 20 resultados
    try:
        # Extraer el nombre del producto usando la clase proporcionada
        name = product.find_element(By.CLASS_NAME, "poly-component__title").text
        
        # Extraer el precio usando la clase correspondiente
        try:
            price = product.find_element(By.CLASS_NAME, "andes-money-amount").text
        except:
            price = "No disponible"
        
        # Añadir datos a la lista
        watch_data.append({"Nombre": name, "Precio": price})
    except Exception as e:
        print("Error al obtener los detalles del producto:", e)

# Cierra el navegador
driver.quit()

# Crear un DataFrame con los datos extraídos
print("Creando dataframe")
df_selenium = pd.DataFrame(watch_data)
df_selenium['Nombre'] = df_selenium['Nombre'].str.replace(',', '')
# Limpiar la columna de precios eliminando el carácter '\n' y cualquier espacio extra
df_selenium['Precio'] = df_selenium['Precio'].str.replace('\n', '').str.strip()

# Guardar el DataFrame en un archivo CSV
df_selenium.to_csv("smart_watch_mercadolibre_selenium.csv", index=False, encoding="utf-8", sep=",")

print(df_selenium)

print("***** Terminado *******")

Buscando en el navegador
Extrayendo resultados
Creando dataframe
***** Terminado *******


In [21]:
# Visualizar el dataframe
df_selenium

,Nombre,Precio
0,"Smartwatch Amazfit GTR 3 Pro display de 1.45"" ...",$1.099.900
1,SmartBand Xiaomi Smart Band 8 Active M2302B1 n...,$149.875
2,Samsung Galaxy Fit 3 Smartwatch Sport Gris Osc...,$333.167
3,Smart Watch Reloj Inteligente Z78 Ultra | Mult...,$92.970
4,Smartwatch Reloj Inteligente Deportivo Linkon ...,$199.990
5,Reloj Xiaomi Smart Band 8 Active Pink Pink Glo...,$178.900
6,Xiaomi Redmi Watch 5 Active Con Hyperos Bateri...,$203.900
7,"Smartwatch Amazfit GTR 3 Pro display de 1.45"" ...",$1.099.900
8,Reloj Inteligente W29 Pro Nuevo Smartwatch Ser...,$199.900
9,Reloj Inteligente Ip67 Impermeable Con Linterna,$220.000


In [ ]:
# Filtrar el dataframe por la marca que nos interese
print("Filtro de Xiaomi")
print(df_selenium[df_selenium["Nombre"].str.contains("Xiaomi")])

,Nombre,Precio
1,SmartBand Xiaomi Smart Band 8 Active M2302B1 n...,$149.875
5,Reloj Xiaomi Smart Band 8 Active Pink Pink Glo...,$178.900
6,Xiaomi Redmi Watch 5 Active Con Hyperos Bateri...,$203.900
10,SmartBand Xiaomi Smart Band 8 Active M2302B1 n...,$149.875
11,Xiaomi Redmi Watch 3 Active Smartband Negro Pa...,$208.970
17,"Xiaomi Series 8 Mi Band 8 display de 1.62"" con...",$192.900
18,Pulsera Inteligente Xiaomi Mi Band 9 Version G...,$223.400


# Beatifulsoup

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

print("****** Iniciando ******** ")

# Define la URL de búsqueda en Mercado Libre
url = "https://listado.mercadolibre.com.co/smart-watch"

# Realiza la solicitud a la página
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)

# Analiza el contenido HTML de la página
soup = BeautifulSoup(response.text, 'html.parser')

# Listas para almacenar los datos
names = []
prices = []

# Encuentra todos los contenedores de productos
print("Listando productos")
products = soup.find_all("li", class_="ui-search-layout__item")

# Extrae el nombre y precio de cada producto
for product in products[:20]:  # Limitamos a los primeros 20 resultados
    # Extrae el nombre del producto
    name_tag = product.find("h2", class_="poly-component__title")
    if name_tag:
        names.append(name_tag.text.strip())

    # Extrae el precio del producto
    price_tag = product.find("span", class_="andes-money-amount")
    if price_tag:
        prices.append(price_tag.text.strip())

# Crea un DataFrame con los datos extraídos
print("Ceando dataframe...")
df_bs4 = pd.DataFrame({
    "Nombre": names,
    "Precio": prices
})

df_bs4['Nombre'] = df_bs4['Nombre'].str.replace(',', '')

print("Creando archivo")

df_bs4.to_csv("smart_watch_mercadolibre.csv", index=False, encoding="utf-8")

print(df_bs4)

print("***** Terminado *****")


****** Iniciando ******** 
Listando productos
Ceando dataframe...
Creando archivo
***** Terminado *****


In [27]:
# Muestra el DataFrame
df_bs4

,Nombre,Precio
0,Pulsera Inteligente Xiaomi Mi Band 9 Version G...,$189.890
1,Pulsera Compatible Con Xiaomi Mi Band 3,$8.000
2,Samsung Galaxy Fit 3 Smartwatch Sport Gris Osc...,$210.900
3,Reloj Xiaomi Smart Band 8 Active Pink Pink Glo...,$100.900
4,Xiaomi Redmi Watch 5 Active Con Hyperos Bateri...,$163.120
5,"Smartwatch Amazfit GTR 3 Pro display de 1.45""...",$549.950
6,Smartwatch Redmi Watch 3 Active Color De La Ca...,$230.000
7,SmartBand Xiaomi Smart Band 8 Active M2302B1 n...,$114.500
8,Xiaomi Redmi Watch 3 Active Smartband Negro Pa...,$208.970
9,Redmi Watch 4 Obsidian Black - Negro Version G...,$334.900
